In [2]:
import cv2
import numpy as np
from pathlib import Path

In [3]:
root = Path().resolve().parent.parent

In [4]:
feature_path = root / 'datasets' / 'ccd' / 'vgg16_features'
file_class = 'positive'
np_file = '000004.npz'

In [5]:
np_feature = np.load(feature_path / file_class / np_file)

In [6]:
np_feature['data'].shape

(50, 20, 4096)

In [7]:
np_feature

NpzFile '/home/don/Projects/misc/major-project-works/datasets/ccd/vgg16_features/positive/000004.npz' with keys: data, det, labels, ID

In [8]:
np_feature['det'].shape

(50, 19, 6)

In [9]:
video_path = root / 'datasets' / 'ccd' /'videos'
videofile = '000004.mp4'

In [10]:
cap = cv2.VideoCapture(str(video_path / file_class / videofile))

In [4]:
frame_index = 0
fps = 10  # Video frame rate

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("End of video reached or error reading frame.")
        break

    # Draw bounding boxes for the current frame
    if frame_index < len(np_feature['det']):
        for det in np_feature['det'][frame_index]:
            x1, y1, x2, y2, prob, cls = det

            color = (0, 255, 0) if cls == 1 else (0, 0, 255)  # Green for class 1, red otherwise
            thickness = 2
            frame = cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, thickness)
            label = f"Class: {int(cls)}, Prob: {prob:.2f}"
            frame = cv2.putText(frame, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

    # Display the frame
    cv2.imshow("Video with Bounding Boxes", frame)

    # Wait for the correct time to match frame rate
    key = cv2.waitKey(0)  # Wait indefinitely for keypress
    if key & 0xFF == ord('q'):  # Quit on 'q'
        break
    elif key & 0xFF == ord('n'):  # Next frame on 'n'
        frame_index += 1
    elif key & 0xFF == ord('p'):  # Previous frame on 'p'
        frame_index = max(0, frame_index - 1)
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)

cap.release()
#cv2.destroyAllWindows()

NameError: name 'cap' is not defined

## Object Depth

In [4]:
data_path = root / 'sample'

In [ ]:
frame_index = 0
fps = 10  # Video frame rate

cap = cv2.VideoCapture(data_path / '000001.mp4')
np_feature = np.load(data_path / '000001.npz')
obj_depth = np.load(data_path / '000001_obj_depth.npy')

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("End of video reached or error reading frame.")
        break

    # Draw bounding boxes for the current frame
    if frame_index < len(np_feature['det']):
        for obj in range(len(np_feature['det'][frame_index])):

            x1, y1, x2, y2, prob, cls = np_feature['det'][frame_index][obj]

            depth = obj_depth[frame_index][obj]

            color = (0, 255, 0) if cls == 1 else (0, 0, 255)  # Green for class 1, red otherwise
            thickness = 2
            frame = cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, thickness)
            label = f"Class: {int(cls)}, Prob: {prob:.2f}, o:{obj}"
            frame = cv2.putText(frame, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
            frame = cv2.putText(frame, f'{obj}, D:{depth}', (int((x1+x2)//2), int((y1+y2)//2)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

    # Display the frame
    cv2.imshow("Video with Bounding Boxes", frame)

    # Wait for the correct time to match frame rate
    key = cv2.waitKey(0)  # Wait indefinitely for keypress
    if key & 0xFF == ord('q'):  # Quit on 'q'
        break
    elif key & 0xFF == ord('n'):  # Next frame on 'n'
        frame_index += 1
    elif key & 0xFF == ord('p'):  # Previous frame on 'p'
        frame_index = max(0, frame_index - 1)
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)

cap.release()
#cv2.destroyAllWindows()

In [8]:
print(np.load(data_path / '000001_obj_depth.npy').shape)

(50, 19)


In [7]:
obj_depth = np.load(data_path / '000001_obj_depth.npy')
print(obj_depth)

[[4.61861801 1.12790048 2.02997923 5.60566711 0.98698783 1.33653867
  1.30505061 1.13010597 5.60566711 1.84796429 2.11609077 4.6310358
  5.60566711 2.11609077 4.6310358  1.30505061 0.98698783 4.61861801
  1.12790048]
 [4.20569658 1.04874396 2.23657274 1.00774372 1.19360423 5.33726168
  1.30057645 1.07238793 5.30112267 1.94002485 2.0071907  4.21110916
  5.30112267 2.0071907  4.21110916 1.30057645 1.00774372 2.23657274
  1.04874396]
 [4.6365242  4.19828987 1.02562785 0.98082983 1.28403401 0.97889566
  1.14648139 1.08370805 2.15707302 4.6365242  1.94211292 4.19828987
  4.6365242  4.19828987 1.05427313 1.16298008 1.02562785 4.6365242
  1.28403401]
 [5.22641897 4.18880939 0.98334718 0.9685508  1.21118665 0.95699358
  1.10936213 1.00849879 2.25619364 5.22641897 2.01458549 4.18880939
  5.22641897 4.18880939 1.02951849 1.09580755 4.18880939 1.21118665
  5.22641897]
 [1.03588021 1.15316689 0.8687529  0.97726136 1.09239841 4.2670784
  4.28015137 1.14480591 1.19417882 2.12360811 4.2670784  1.1531